In [ ]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc
import helper
import tx
import script

In [ ]:
from tx import Tx
from io import BytesIO
raw_tx = ('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600')
stream = BytesIO(bytes.fromhex(raw_tx))
transaction = Tx.parse(stream)
print(transaction.fee() >= 0)

In [ ]:
transaction

In [ ]:
transaction.fee()

In [ ]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
print(point.verify(z, signature))

In [ ]:
from helper import hash256
modified_tx = bytes.fromhex('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000001976a914a802fc56c704ce87c42d7c92eb75e7896bdc41ae88acfeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac1943060001000000')
h256 = hash256(modified_tx)
z = int.from_bytes(h256, 'big')
print(hex(z))

In [ ]:
from helper import hash256
modified_tx = bytes.fromhex('0100000001713f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000001976a914a802fc56c704ce87c42d7c92eb75e7896bdc41ae88acfeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac1943060001000000')
# 在 tx 中自己改了一个数字，然后结果的 hash 就会差别很大。
h256 = hash256(modified_tx)
z = int.from_bytes(h256, 'big')
print(hex(z))

In [ ]:
h256

In [ ]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
point.verify(z, signature)

### Exercise 1

Write the `sig_hash` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sig_hash`

In [ ]:
# Exercise 1

reload(tx)
run(tx.TxTest("test_sig_hash"))

### Exercise 2

Write the `verify_input` method for the `Tx` class. You will want to use the `TxIn.script_pubkey()`, `Tx.sig_hash()` and `Script.evaluate()` methods.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_verify_p2pkh`

In [ ]:
# Exercise 2

reload(tx)
run(tx.TxTest("test_verify_p2pkh"))

In [ ]:
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx
prev_tx = bytes.fromhex('0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299')
prev_index = 13
tx_in = TxIn(prev_tx, prev_index)
tx_outs = []
change_amount = int(0.33*100000000)
change_h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
change_script = p2pkh_script(change_h160)
change_output = TxOut(amount=change_amount, script_pubkey=change_script)
target_amount = int(0.1*100000000)
target_h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)
tx_obj = Tx(1, [tx_in], [change_output, target_output], 0, True)
print(tx_obj)

In [ ]:
SIGHASH_ALL = 1
SIGHASH_ALL.to_bytes(1, 'big')

In [ ]:
a = 1
a.to_bytes(1, 'big')

In [ ]:
from ecc import PrivateKey
from helper import SIGHASH_ALL
z = transaction.sig_hash(0)
private_key = PrivateKey(secret=8675309)
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sec = private_key.point.sec()
script_sig = Script([sig, sec])
transaction.tx_ins[0].script_sig = script_sig 
print(transaction.serialize().hex())

In [ ]:
from ecc import PrivateKey
from helper import hash256, little_endian_to_int
secret = little_endian_to_int(hash256(b'Jimmy Song secret'))
private_key = PrivateKey(secret)
print(private_key.point.address(testnet=True))

### Exercise 3

Write the `sign_input` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sign_input`

In [ ]:
# Exercise 3

reload(tx)
run(tx.TxTest("test_sign_input"))

### Exercise 4

Create a testnet transaction that sends 60% of a single UTXO to `mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv`. The remaining amount minus fees should go back to your own change address. This should be a 1 input, 2 output transaction.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [ ]:
# Exercise 4

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# create 1 TxIn and 2 TxOuts
# 1 of the TxOuts should be back to your address
# the other TxOut should be to this address
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'

prev_tx = bytes.fromhex('c52af070db39a6a2e1c811aa4c10299019f9427302442376a82c3927e389fa87')
prev_index = 1
tx_in = TxIn(prev_tx, prev_index)
tx_outs = []

change_amount = int(0.0003 * 100000000)
change_h160 = decode_base58('mze1vFH1wdMWSTeuqUL7SFyN9V1jFhC5kc')
change_script = p2pkh_script(change_h160)
change_output = TxOut(amount=change_amount, script_pubkey=change_script)

target_amount = int(0.0006 * 100000000)
target_h160 = decode_base58(target_address)
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)
tx_obj = Tx(1, [tx_in], [change_output, target_output], 0, True)
print(tx_obj)
print(tx_obj.fee())
# get the private key from the exercise in Chapter 4
# change address should be the address generated from Chapter 4

# get the prev_tx and prev_index from the transaction where you got
# some testnet coins
# create a transaction input for the previous transaction with
# the default ScriptSig and sequence

# target amount should be 60% of the output amount
# set the fee to some reasonable amount
# change amount = amount from the prev tx - target amount - fee

# create a transaction output for the target amount and address
# create a transaction output for the change amount and address
# create the transaction object

# sign the one input in the transaction object using the private key
# print the transaction's serialization in hex
# broadcast at http://testnet.blockchain.info/pushtx

In [ ]:
tx_obj.serialize().hex()

In [ ]:
# Exercise 9

from ecc import PrivateKey
from helper import hash256, little_endian_to_int

# select a passphrase here, add your email address into the passphrase for security
# passphrase = b'your@email.address some secret only you know'
# secret = little_endian_to_int(hash256(passphrase))
# create a private key using your secret
# print an address from the public point of the private key with testnet=True

passphrase = b'leeyy do not go gentle into that good night'
secret = little_endian_to_int(hash256(passphrase))
pk = PrivateKey(secret)
adds = pk.point.address(compressed=True, testnet=True)

In [ ]:
tx_obj.sign_input(0, pk)# 需要签名才可以

In [ ]:
tx_obj.serialize().hex()

In [ ]:
tx_obj

### Exercise 5

Advanced: get some more testnet coins from a testnet faucet and create a 2 input, 1 output transaction. 1 input should be from the faucet, the other should be from the previous exercise, the output can be your own address.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [1]:
# Exercise 5

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# Create 2 TxIns, 1 from the Exercise 4 and 1 from a testnet faucet
# Creat 1 TxOut to the address above
# target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
target_address = 'mkHS9ne12qx9pS9VojpwU5xtRd4T7X7ZUt'  #  SEND THE TESTNET COINS BACK https://testnet-faucet.mempool.co/

# get the private key from the exercise in Chapter 4

# get the prev_tx and prev_index from the transaction where you got
# some testnet coins
# create the first transaction input with the default ScriptSig and
# sequence
# get the prev_tx and prev_index from the transaction in Exercise 4
# create the second transaction input with the default ScriptSig and
# sequence
prev_tx1 = bytes.fromhex('cd3068eb082f86534b95c3912e1afd400c6370fb3c596a7aab3cf0f1f939222f')  # tcoins from faucet
prev_index1 = 1

prev_tx2 = bytes.fromhex('5e5798c6bd66246283a164aae1869484bc1130e71abd0db9b1de9bdc5c26cdcb')
prev_index2 = 0

tx_in1 = TxIn(prev_tx1, prev_index1)
tx_in2 = TxIn(prev_tx2, prev_index2)
tx_ins = [tx_in1, tx_in2]

# set the fee to some reasonable amount
# target amount should be the sum of the inputs - fee

# create a transaction output for the amount and address
target_amount = int(0.0012 * 100000000)
target_h160 = decode_base58(target_address)
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)
tx_outs = [target_output]

tx_obj = Tx(1, tx_ins, tx_outs, 0, True)
print(tx_obj)

# sign the first input using the private key
# sign the second input using the private key 
# print the transaction's serialization in hex
# broadcast at http://testnet.blockchain.info/pushtx

tx: 755c7962cc502760b4ca11e094978a7247c340180b4cf9c34de94f352dfadd06
version: 1
tx_ins:
cd3068eb082f86534b95c3912e1afd400c6370fb3c596a7aab3cf0f1f939222f:1
5e5798c6bd66246283a164aae1869484bc1130e71abd0db9b1de9bdc5c26cdcb:0
tx_outs:
119999:OP_DUP OP_HASH160 344a0f48ca150ec2b903817660b9b68b13a67026 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0


In [2]:
tx_obj.fee()

10000

In [3]:
tx_obj.serialize().hex()

'01000000022f2239f9f1f03cab7a6a593cfb70630c40fd1a2e91c3954b53862f08eb6830cd0100000000ffffffffcbcd265cdc9bdeb1b90dbd1ae73011bc849486e1aa64a183622466bdc698575e0000000000ffffffff01bfd40100000000001976a914344a0f48ca150ec2b903817660b9b68b13a6702688ac00000000'

In [4]:
# Exercise 9

from ecc import PrivateKey
from helper import hash256, little_endian_to_int

# select a passphrase here, add your email address into the passphrase for security
# passphrase = b'your@email.address some secret only you know'
# secret = little_endian_to_int(hash256(passphrase))
# create a private key using your secret
# print an address from the public point of the private key with testnet=True

passphrase = b'leeyy do not go gentle into that good night'
secret = little_endian_to_int(hash256(passphrase))
pk = PrivateKey(secret)
adds = pk.point.address(compressed=True, testnet=True)

In [5]:
tx_obj.sign_input(0, pk)

True

In [6]:
tx_obj.sign_input(1, pk)

True

In [7]:
tx_obj.sign_input(2, pk)

IndexError: list index out of range

In [8]:
tx_obj.serialize().hex()

'01000000022f2239f9f1f03cab7a6a593cfb70630c40fd1a2e91c3954b53862f08eb6830cd010000006b483045022100fec49a12d15193eee9cfb68ea6ca273add2114297ac349cdb82272022bda632102205deb51a4ef1b6c95cb87bf87ca888cf2f7e5f4c36f6dab6536417ccccf6170fc012102355b100e729b2c84b96fbd6c0419725d5b59e233c9e3c4f2df6b057fd78ced59ffffffffcbcd265cdc9bdeb1b90dbd1ae73011bc849486e1aa64a183622466bdc698575e000000006a47304402204b9df1b30c585ecaf4525b5ee57d4b7d3fdc827ff569192ca3d8b9b930ca2aed0220718a6a247473e1a65f0185cd5822c9533379d009992830f1abd9473b749d0af8012102355b100e729b2c84b96fbd6c0419725d5b59e233c9e3c4f2df6b057fd78ced59ffffffff01bfd40100000000001976a914344a0f48ca150ec2b903817660b9b68b13a6702688ac00000000'

In [9]:
tx_obj

tx: 870e5ad05326b247342221012b7852ec47b6b418750df58ac5d06a80fd6bbd56
version: 1
tx_ins:
cd3068eb082f86534b95c3912e1afd400c6370fb3c596a7aab3cf0f1f939222f:1
5e5798c6bd66246283a164aae1869484bc1130e71abd0db9b1de9bdc5c26cdcb:0
tx_outs:
119999:OP_DUP OP_HASH160 344a0f48ca150ec2b903817660b9b68b13a67026 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0